<a href="https://colab.research.google.com/github/kallviktor/RandomInterpolationGAN/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/kmualim/DCGAN-Keras-Implementation/blob/master/dcgan-mnist.py

In [179]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Flatten, Reshape 
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D 
from keras.layers import LeakyReLU, Dropout 
from keras.layers import BatchNormalization 
from keras.optimizers import Adam, RMSprop
from keras.initializers import RandomNormal, Zeros
from keras.datasets import mnist
import matplotlib.pyplot as plt 
import sys 
import numpy as np

class GAN(object): 
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28 
        self.channel=1
        self.img_shape = (self.img_rows, self.img_cols, self.channel)
        optimizer = Adam(0.0002, 0.5)
     
    def build_discriminator(self):
        model = Sequential()
        depth = 32 
        dropout=0.25 
        input_shape = (self.img_rows, self.img_cols, self.channel)
        model.add(Conv2D(depth*1, 3, strides=2, input_shape=input_shape, padding='same', kernel_initializer='random_uniform'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))
        model.add(Conv2D(depth*2, 3, strides=2, padding='same',kernel_initializer='random_uniform'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))
        model.add(Conv2D(depth*4, 3, strides=2, padding='same',kernel_initializer='random_uniform'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))
        model.add(Conv2D(depth*8, 3, strides=2, padding='same',kernel_initializer='random_uniform'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))

        # Each MNIST input = 28 X 28 X 1, depth = 1
        # Each Output = 14 X 14 X 1, depth = 64 
        # Model has 4 convolutional layer, each with a dropout layer in between 

        # Output 
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.summary()
        
        img = Input(shape=(self.img_shape))
        validity = model(img)
      
        return Model(img, validity) 

    # generator takes noise as input and generates imgs
                
    def build_generator(self):
        generator = Sequential() 
        dropout = 0.4 
        depth = 128
        dim = 7

      # In: 100 
      # Out: dim X dim X depth 
        
        #generator.add(Dense(dim*dim*depth, input_dim=100))
        generator.add(Input(shape=(1,)))
        generator.add(Dense(dim*dim*depth, input_dim=1))
        #generator.add(Dense(dim*dim*depth)
        generator.add(Activation('relu'))
        generator.add(Reshape((dim, dim, depth)))
        generator.add(UpSampling2D())
      #generator.add(Dropout(dropout))

      # In: dim X dim X depth
      # Out: 2*dim X 2*dim X depth/2 

        generator.add(Conv2D(depth, 3, padding='same'))
        generator.add(BatchNormalization(momentum=0.9))
        generator.add(Activation('relu'))
        generator.add(UpSampling2D())
        generator.add(Conv2D(int(depth/2), 3, padding='same'))
        generator.add(BatchNormalization(momentum=0.9))
        generator.add(Activation('relu'))
     

        # Out : 28 X 28 X 1 grayscale image [0.0, 1.0] per pix
        generator.add(Conv2D(1,3,padding='same'))
        generator.add(Activation('tanh'))
        generator.summary()
      
        #noise = Input(shape=(100,))
        noise = Input(shape=(1,))
        img = generator(noise)
        return Model(noise, img)
    
    # Build and compile discriminator
    def DM(self):
        optimizer = Adam(0.0002, 0.5)
        DM = self.build_discriminator()
        DM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return DM    

class dcgan(object):
    def __init__(self):
        self.img_rows=28
        self.img_cols=28
        self.channels=1

        # building the generator 
        self.GAN = GAN()
        self.DM = self.GAN.DM()
        self.generator = self.GAN.build_generator()


        #z = Input(shape=(100,))
        z = Input(shape=(1,))
        img = self.generator(z)
        self.DM.trainable = False
        valid = self.DM(img)
    
        self.combined = Model(z, valid)
        optimizer = Adam(0.0002, 0.5)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
        # training input 
        # To change dataset, place dataset below 
        (self.x_train, _), (_,_) = mnist.load_data()
        self.x_train = self.x_train/127.5 -1.
        self.x_train = np.expand_dims(self.x_train, axis=3) 
        #x_train = x_train/127.5 -1. 
        #x_train = np.expand_dims(x_train, axis=3)
        self.n_samples = 25
        #self.noise_dim = 100
        self.noise_dim = 1
    
  # method to generate noise 
    def gennoise(self,batch_size, noise_dim): 
        x = np.random.normal(0, 1.0, (batch_size, self.noise_dim))
        return x
    
    def plt_imgs(self,epoch): 
        noise = self.gennoise(self.n_samples, self.noise_dim)
        fake_imgs = self.generator.predict(noise)
        fake_imgs = 0.5 * fake_imgs + 0.5
  
        fig,axs = plt.subplots(5,5)
        count = 0 
        for i in range(5): 
            for j in range(5): 
                axs[i,j].imshow(fake_imgs[count, :, :, 0], cmap='gray')
                axs[i,j].axis('off')
                count+=1
      
        fig.savefig("mnist_%d.png" % epoch)
        plt.close()
  
  
    def train(self,n_epochs, batch_size):
        train_hist={}
        train_hist['D_losses']=[]
        train_hist['G_losses']=[]
        print("Start")
        true_labels=np.ones((batch_size,1))
        gen_gene_labels=np.zeros((batch_size,1))
    
        for epoch in range(n_epochs):
        
            index = np.random.randint(0, self.x_train.shape[0], batch_size)
            images = self.x_train[index]
      
            #noise_data = self.gennoise(batch_size, 100)
            noise_data = self.gennoise(batch_size, 1)
            gen_imgs = self.generator.predict(noise_data)
      
      
            d_loss = self.DM.train_on_batch(images, true_labels)
    
            d_loss_generated = self.DM.train_on_batch(gen_imgs, gen_gene_labels)
      
            total_d_loss = 0.5 * np.add(d_loss, d_loss_generated)
      
            train_hist['D_losses'].append(total_d_loss[0])
        
            #noise_data = self.gennoise(batch_size, 100)
            noise_data = self.gennoise(batch_size, 1)
            y1 = np.ones((batch_size, 1))    
      
            g_loss = self.combined.train_on_batch(noise_data, y1)

            train_hist['G_losses'].append(g_loss)
            print (' Epoch:{}, G_loss: {}, D_loss:{}'.format(epoch+1, g_loss, total_d_loss[0]))
      
            if epoch%50==0:
                self.plt_imgs(epoch)
      
        return train_hist
  
    def plotting_imgs(self,epoch): 
        #noise = self.gennoise(25,100)
        noise = self.gennoise(25,1)
        fake_imgs = self.generator.predict(noise)
        fake_imgs = 0.5 * fake_imgs + 0.5
    
        fig,axs = plt.subplots(5,5)
        count = 0 
        for i in range(5): 
            for j in range(5): 
                axs[i,j].imshow(fake_imgs[count, :, :, 0], cmap='gray')
                axs[i,j].axis('off')
                count+=1

if __name__ == '__main__': 
    
    train_hist = mnist_dcgan.train(10, batch_size=32)

Start
 Epoch:1, G_loss: 1.3526071310043335, D_loss:0.2624758332967758
 Epoch:2, G_loss: 1.4007810354232788, D_loss:0.41811051964759827
 Epoch:3, G_loss: 1.4643970727920532, D_loss:0.3203582763671875
 Epoch:4, G_loss: 1.1368889808654785, D_loss:0.17498047649860382
 Epoch:5, G_loss: 1.0625934600830078, D_loss:0.25878091901540756
 Epoch:6, G_loss: 1.0037124156951904, D_loss:0.21011468768119812
 Epoch:7, G_loss: 1.2775890827178955, D_loss:0.10924840345978737
 Epoch:8, G_loss: 1.0839972496032715, D_loss:0.21700330823659897
 Epoch:9, G_loss: 0.8469861149787903, D_loss:0.10762898623943329
 Epoch:10, G_loss: 1.1944339275360107, D_loss:0.17658867686986923


In [124]:
def kernel(h, alpha=1, beta=1):
    
    k = np.exp(-beta*np.abs(h)**alpha)
    
    return k

In [125]:
def cov_func(T, t, s):
    
    k_hat = kernel(t-s)+ \
            (kernel(T)*(kernel(T-s)*kernel(t)+kernel(s)*kernel(T-t)))/(1-kernel(T)**2) - \
            (kernel(T-s)*kernel(T-t)+kernel(s)*kernel(t))/(1-kernel(T)**2)
    
    return k_hat

In [126]:
def mean_func(T, t, z0, zT):
    
    mu_hat = (z0*(kernel(t)-kernel(T-t)*kernel(T))+zT*(kernel(T-t)-kernel(t)*kernel(T)))/(1-kernel(T)**2)
    
    return mu_hat

In [127]:
def cov_mat(T, N):
    
    C = np.zeros([N+1,N+1])
    
    for i in range(N+1):
        for j in range(N+1):
            
            t = i*T/N
            s = j*T/N
            
            C[i,j] = cov_func(T,t,s)
    
    return C

In [128]:
def mu_vec(T, N, z0=0, zT=1):
    
    mu = np.zeros(N+1)
    
    for i in range(N+1):
        
        t = i*T/N
        mu[i] = mean_func(T,t,z0,zT)
    
    return mu

In [129]:
def weight_func(z,G,D):
    
    f = D.predict(G.predict(z))/(1-D.predict(G.predict(z)))
    
    return f

In [130]:
def gen_path(T,N,z0=0,zT=1):
    
    path = np.random.multivariate_normal(mu_vec(T,N,z0,zT),cov_mat(T,N))
    
    return path

In [131]:
D = mnist_dcgan.DM
G = mnist_dcgan.generator

M = 10
N = 10
n = N
T = 1
dT = T/N

trajectory = np.zeros(N)

z0 = 0
zT = 1

trajectory[0] = z0

for j in range(N-1):
    
    weights = np.zeros(M)
    particles = np.zeros(M)
    #print(particles)
    
    for i in range(M):
        print(i)
        path = gen_path(T,n,z0,zT)
        #print(path)
        #print(path.shape)
        particles[i] = path[i]
        print(path[1])
        weights[i] = weight_func(path[1],G,D)

    z_hat = particles[np.argmax(weights)]
    z0 = z_hat
    
    trajectory[j+1] = z0
    
    n = n-1
    T = T - dT

0
-0.4530962156786065


IndexError: list index out of range

In [132]:
gen_path(1,10)

array([-1.49211934e-09,  3.20248814e-01,  1.20922446e-01,  2.50587956e-01,
        3.30852635e-01,  1.10820890e+00,  1.11643903e+00,  1.60697129e+00,
        1.77681397e+00,  1.39915580e+00,  9.99999975e-01])

In [135]:
weight_func(path[1],G,D)

IndexError: list index out of range

In [137]:
D.predict(G.predict(path[1]))

IndexError: list index out of range

In [139]:
test_path = gen_path(1,10,0,1)

In [189]:
test_path
mnist_dcgan.generator.input_shape

(None, 100)